In [1]:
import re

In [34]:
# To create getter and setter functions for the component information, we'll define two functions:
# 1. get_components: This function will parse the .kicad_pcb file and return a list of components with their details.
# 2. set_components: This function will take a list of components with updated positions/rotations and modify the .kicad_pcb file accordingly.


def get_components(file_data):
    """
    Parses the .kicad_pcb file data and extracts component information along with the designator.

    Args:
    file_data (str): Contents of the .kicad_pcb file.

    Returns:
    list of dicts: List of components with details (uuid, position, rotation, designator).
    """
    # Regex to match component footprint and extract UUID, position, rotation, and designator
    component_regex = re.compile(
        r'\(footprint\s+"[^"]+"\s+\(layer\s+"[^"]+"\)\s*\(tstamp\s+([^\s]+)\)\s*\(at\s+([0-9.-]+)\s+([0-9.-]+)\s*([0-9.-]*)\).*?\(fp_text\s+reference\s+"([^"]+)"',
        re.MULTILINE | re.DOTALL
    )

    components = component_regex.findall(file_data)
    parsed_components = [
        {'uuid': uuid, 'position': {'x': float(x), 'y': float(y)}, 'rotation': float(rot) if rot else 0.0, 'designator': designator}
        for uuid, x, y, rot, designator in components
    ]
    return parsed_components

def set_components(file_data, updated_components):
    """
    Updates the .kicad_pcb file data with new positions and rotations for components.
    Also updates pad rotations by adding the component's rotation to the original pad rotation.

    Args:
    file_data (str): Original contents of the .kicad_pcb file.
    updated_components (list of dicts): List of components with updated details (uuid, new position, new rotation).

    Returns:
    str: Updated contents of the .kicad_pcb file.
    """
    for component in updated_components:
        uuid = component['uuid']
        new_x = component['position']['x']
        new_y = component['position']['y']
        new_rot = component['rotation']

        # Update the position and rotation of the component in the file data
        file_data = re.sub(
            rf'(\(footprint\s+"[^"]+"\s+\(layer\s+"[^"]+"\)\s*\(tstamp\s+{uuid}\)\s*)\(at\s+([0-9.-]+)\s+([0-9.-]+)\s*([0-9.-]*)\)',
            rf'\1(at {new_x} {new_y} {new_rot})',
            file_data
        )

        # Regex for finding and updating pad rotations
        pad_regex = rf'(\(pad\s+[^\(]*\(at\s+([0-9.-]+)\s+([0-9.-]+)\s*)([0-9.-]+)(\)\s+\(size)'
        def pad_rotation_match(match):
            original_pad_rot = float(match.group(4))
            new_pad_rot = original_pad_rot + new_rot
            return f"{match.group(1)}{new_pad_rot}{match.group(5)}"
        file_data = re.sub(pad_regex, pad_rotation_match, file_data, flags=re.MULTILINE)

    return file_data



In [35]:
# import pcb file
project_path = "/Users/narayanpowderly/Documents/atopile-workspace/blurp/elec/layout/blurp.kicad_pcb"
kicad_pcb_data = open(project_path, 'r').read()

In [36]:
components = get_components(kicad_pcb_data)

# Let's say we want to update the position and rotation of the first component
# components[0]['position']['x'] += 1000  # Move the x position by 10mm
# components[0]['rotation'] = 45  # Rotate the component to 45 degrees
# rotate all components by 35 degrees
for component in components:
    component['rotation'] += 35

# Updating the file with the new component data
updated_file_data = set_components(kicad_pcb_data, components)

# Write the updated file data to a new file
with open(project_path, 'w') as f:
    f.write(updated_file_data)

# Display the first part of the updated file data to verify the change
# print(updated_file_data[:1000])  # Print first 1000 characters of the updated file data
# components
updated_components = get_components(updated_file_data)

updated_components

[{'uuid': '04b151b8-8c42-4843-aab0-15ca151b6d42',
  'position': {'x': 153.14, 'y': 85.22},
  'rotation': 35.0,
  'designator': 'C4'},
 {'uuid': '1d11faa8-d6ee-455b-896a-ac23a1193ad6',
  'position': {'x': 153.16, 'y': 95.25},
  'rotation': 35.0,
  'designator': 'R2'},
 {'uuid': '22b0138b-29af-4ee3-b83c-af666ef1e228',
  'position': {'x': 153.16, 'y': 90.24},
  'rotation': 35.0,
  'designator': 'R1'},
 {'uuid': '2edaabac-758e-4607-a9d3-16d75b47d3d8',
  'position': {'x': 147.48, 'y': 95.76},
  'rotation': 35.0,
  'designator': 'C2'},
 {'uuid': '50bd3a70-33f5-4ee0-96bf-95911c3296af',
  'position': {'x': 140.73, 'y': 92.19},
  'rotation': 35.0,
  'designator': 'U3'},
 {'uuid': '5c30d0e1-14ff-4194-ae79-53e235712b8c',
  'position': {'x': 147.48, 'y': 85.74},
  'rotation': 35.0,
  'designator': 'C3'},
 {'uuid': '66c82c11-81c9-4950-97a7-b73b11f7db89',
  'position': {'x': 140.625, 'y': 96.87},
  'rotation': 35.0,
  'designator': 'U2'},
 {'uuid': '7e52176e-bfaf-45ff-8a45-8b8d6549027a',
  'position